In [1]:
! pip install xgboost
! pip install pandas
! pip install scikit-learn
! pip install pyarrow

  Using cached xgboost-1.7.5-py3-none-manylinux2014_x86_64.whl (200.3 MB)
  Using cached scipy-1.10.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.4 MB)
  Using cached numpy-1.24.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached pandas-2.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.3 MB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Using cached scikit_learn-1.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.6 MB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached pyarrow-12.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.9 MB)


TypeError: 'SparkXGBRegressorModel' object is not callable

In [ ]:
import xgboost as xgb
bst = xgb.Booster()
# Loading the model saved in previous snippet
bst.load_model("/tmp/xgboost-pyspark-model/model/part-00000")

In [1]:
import pyspark
from pyspark.sql import SparkSession

from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import DoubleType, FloatType, LongType
from pyspark.sql.functions import col, mean, stddev
from pyspark.sql import Window

import pyspark.sql.functions as F
from pyspark.ml import Transformer
from pyspark.sql import DataFrame
import pandas as pd
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable 

from xgboost.spark import SparkXGBRegressor
from pyspark.ml import Pipeline

import xgboost as xgb

In [2]:
spark = SparkSession.builder.appName("check_files").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/18 02:01:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [23]:
data_location1='/cephfs/summary_stats/train/summary_stats_parquet_42_26.parquet'
data_location2='/cephfs/summary_stats/train/summary_stats_parquet_159_25.parquet'

In [27]:
data_og='/cephfs/interpolation/train/parquet_42_26.parquet'

In [28]:
df1=spark.read.format('parquet').load(data_location1)
df1.show(1)

+-----+-----+----+------+------+---+---+------------+----------+------------+----------+----------+----------+---------------+------+
|NumId|Chunk|Mean|StdDev|Median|Min|Max|AvgFirstDiff|AvgSecDiff|StdFirstDiff|StdSecDiff|CountAbove|CountBelow|TotalOutOfRange|target|
+-----+-----+----+------+------+---+---+------------+----------+------------+----------+----------+----------+---------------+------+
+-----+-----+----+------+------+---+---+------------+----------+------------+----------+----------+----------+---------------+------+



In [25]:
class ColumnScaler(Transformer, DefaultParamsReadable, DefaultParamsWritable):
    def _transform(self, df):
        double_cols=[f.name for f in df.schema.fields if isinstance(f.dataType, DoubleType)]
        float_cols=[f.name for f in df.schema.fields if isinstance(f.dataType, FloatType)]
        long_cols=[f.name for f in df.schema.fields if isinstance(f.dataType, LongType)]

        all_numerical=list(set(double_cols+float_cols+long_cols))
        all_numerical.remove('target')
        
        for num_column in all_numerical:
            input_col = f"{num_column}"
            output_col = f"scaled_{num_column}"

            w = Window.partitionBy('NumId')

            mu = mean(input_col).over(w)
            sigma = stddev(input_col).over(w)

            df=df.withColumn(output_col, (col(input_col) - mu)/(sigma))
            
        return df

In [26]:
double_cols=[f.name for f in df1.schema.fields if isinstance(f.dataType, DoubleType)]
float_cols=[f.name for f in df1.schema.fields if isinstance(f.dataType, FloatType)]
long_cols=[f.name for f in df1.schema.fields if isinstance(f.dataType, LongType)]

all_numerical=list(set(double_cols+float_cols+long_cols))
all_numerical.remove('target')

featureArr = [('scaled_' + f) for f in all_numerical]

columns_scaler=ColumnScaler()

va2 = VectorAssembler(inputCols=featureArr, outputCol="features", handleInvalid='skip')

stages=[columns_scaler]+[va2]

In [20]:
features_col="features"
label_name="target"

xgb_regression=SparkXGBRegressor(features_col=features_col, 
                                  label_col=label_name,
                                  random_state=123,
                                  use_gpu=False)


stages.append(xgb_regression)
pipeline=Pipeline(stages=stages)

model=pipeline.fit(df1)

[03:15:50] task 0 got new rank 0                                    (0 + 1) / 1]


[03:15:51] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.



In [8]:
model.write().overwrite().save('Saved_Models/model_1')

In [9]:
df2=spark.read.format('parquet').load(data_location2)
df2.show(1)

+-----+-----+------------------+------------------+------+----+-----+--------------------+--------------------+------------------+------------------+----------+----------+---------------+------+
|NumId|Chunk|              Mean|            StdDev|Median| Min|  Max|        AvgFirstDiff|          AvgSecDiff|      StdFirstDiff|        StdSecDiff|CountAbove|CountBelow|TotalOutOfRange|target|
+-----+-----+------------------+------------------+------+----+-----+--------------------+--------------------+------------------+------------------+----------+----------+---------------+------+
|  168|    1|106.13888888888889|14.927285360656931| 106.0|81.0|140.0|-0.00694444444444...|0.017361111111111112|2.1085408789060285|1.5914768571721178|         0|         0|              0|     6|
+-----+-----+------------------+------------------+------+----+-----+--------------------+--------------------+------------------+------------------+----------+----------+---------------+------+
only showing top 1 row



In [10]:
double_cols=[f.name for f in df2.schema.fields if isinstance(f.dataType, DoubleType)]
float_cols=[f.name for f in df2.schema.fields if isinstance(f.dataType, FloatType)]
long_cols=[f.name for f in df2.schema.fields if isinstance(f.dataType, LongType)]

all_numerical=list(set(double_cols+float_cols+long_cols))
all_numerical.remove('target')

featureArr = [('scaled_' + f) for f in all_numerical]

columns_scaler=ColumnScaler()

va2 = VectorAssembler(inputCols=featureArr, outputCol="features", handleInvalid='skip')

stages=[columns_scaler]+[va2]

In [11]:
features_col="features"
label_name="target"

xgb_classifier=SparkXGBRegressor(features_col=features_col, 
                                 label_col=label_name,
                                  random_state=123,
                                  use_gpu=False,
                                  xgb_model=model.stages[-1].get_booster())



stages.append(xgb_classifier)
pipeline=Pipeline(stages=stages)

model=pipeline.fit(df2)

[02:04:52] WARNING: ../src/learner.cc:553:                          (0 + 1) / 1]
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[02:04:54] task 0 got new rank 0


[02:04:56] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.



In [12]:
model.write().overwrite().save('Saved_Models/model_2')

23/05/18 02:05:04 WARN TaskSetManager: Stage 28 contains a task of very large size (1159 KiB). The maximum recommended task size is 1000 KiB.


In [13]:
1%20

1

In [15]:
40%20

0